# VOQC: A Verifed Optimizer for Quantum Circuits  <img src="img/umd.jpeg" width="100" align="right">


Welcome to VOQC, a verified optimizer for quantum circuits! This tutorial aims to introduce you to the different optimizations and transformations in VOQC and how to take advantage of them in Qiskit, with VOQC’s interoperability. You can now build a quantum circuit in Qiskit and optimize it through VOQC, with one or more of its unitary optimizations, just like Qiskit’s built-in optimizations. 

This tutorial is divided into two primary sections:
* Passing a Qiskit Circuit Object to VOQC
* Running Individual Unitary Optimizations
  * Not Propagation
  * Cancel Single Qubit Gates
  * Cancel Two Qubit Gates
  * Rotation Merging
  * Hadamard Reduction 

The *Running Individual Unitary Optimizations* section contains five subsections, one for each optimization function in VOQC. These subsections will have a short summary of the optimization itself, a more detailed version of the optimizations can be found in the paper <a href="https://arxiv.org/abs/1912.02250">A Verified Optimizer for Quantum Circuits</a>, and an example of how to use it in Qiskit.  

All of the code referenced in this notebook can be found <a href="https://github.com/inQWIRE/SQIR">here</a>.

Before we can get started, we must import modules from Qiskit and VOQC that we will use in this tutorial. 

**Select the cell below and press *Ctrl+Enter* to import the necessary modules.**

In [ ]:
#Click on this cell and press Ctrl+Enter or run it with the "Run" Button
from qiskit import QuantumCircuit
from interop.voqc import SQIR
from interop.qiskit.voqc_optimization import VOQC
from qiskit.transpiler import PassManager
from qiskit.qasm import pi

## Passing a Qiskit Circuit Object to VOQC

Because of VOQC's interoperability with Cirq and Qiskit, it is now possible to pass a Qiskit circuit object through the VOQC optimizations and receive an optimized Qiskit circuit.

The VOQC transpiler can be called similarly to Qiskit's built-in transpiler passes (e.g. "Commutative Cancellation" and "CX Cancellation"). **VOQC(\[*list of optimizations*\])** can be appended to a pre-defined **Pass Manager** with "list of optimizations" being an optional argument used to run one or more of the unitary optiizations in VOQC (see *Running Individual Unitary Optimizations*). Appending **VOQC()**, without the argument, will run VOQC's primary optimization function, a specific order of the unitary optimizations. 

### Example 1

In our first example, we will pass a basic Qiskit circuit to VOQC without the optional list argument, taking advantage of VOQC's main optimization function.

We first create a four-qubit circuit with 2 NOT gates, 2 hadamard gates, and a CNOT gate. Given this sample circuit **circ**, we define a Pass Manager **pm** to schedule our VOQC transpiler pass. Then, we can append the VOQC transpiler pass to **pm** without an argument, **VOQC()**. Finally, we run our initial circuit **circ** through the primary VOQC optimization and receive and print our optimized circuit **new_circ**.  

We began with a circuit consisting of 5 gates, but have reduced it to just a singular gate. Now, we will go into more detail on the specifics of each individual optimization.


In [ ]:
#Build Circuit with 4 Qubits and 5 Gates (2 NOT, 2 Hadamard, and 1 CNOT)
circ = QuantumCircuit(4)
circ.x(0)
circ.h(1)
circ.x(0)
circ.h(1)
circ.cx(2, 3)
print("Before Optimization:")
print(circ)

#Append VOQC pass without argument to the Pass Manager
pm = PassManager()
pm.append(VOQC())
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

## Running Individual Unitary Optimizations

While appending **VOQC()** to the pass manager runs the primary VOQC optimization function, there are scenarios where you may want to run one or more of the unitary optimizations of VOQC. These five optimizations are *not propagation*, *cancel single qubit gates*, *cancel two qubit gates*, *merge rotations*, and *hadamard reduction*. In this section, we will describe each of the five aforementioned optimizations, including their purpose and how to incorprate them into your compiler pass.

As always, the implementations of these optimizations can be found <a href="https://github.com/inQWIRE/SQIR">here</a> and the paper itself can be found <a href="https://arxiv.org/abs/1912.02250">here</a>. 

### Not Propagation

<p>The "not propagation" optimization cancels two NOT gates (X) that are either adjacent or separated by a circuit that commutes with X. For commuting circuits, the identities below are applied repeatedly to simplify the circuit.</p> 

<p><center>$X q;  H q \equiv H q;  Z q$</center></p>
<p><center>$X q;  Rz(k) q \approx Rz(2-k) q;  X q$</center></p>
<p><center>$X q_{1};  CNOT q_{1} q_{2} \equiv CNOT q_{1} q_{2};  X q_{1};  X q_{2}$</center></p>
<p><center>$X q_{2};  CNOT q_{1} q_{2} \equiv CNOT q_{1} q_{2};  X q_{2}$</center></p>


#### Example 2

This time, instead of using the transpiler pass without the optional list argument, we will pass a list of one element to VOQC. The optional list argument to invoke the *not propagation* optimization is **not_propagation**, so we append the following to the pass manager: **VOQC(["not_propagation"])**. 
    
To test this, we will create a two-qubit circuit, consisting of 2 X gates and one CX gate. Once this is passed to VOQC, the leftmost X gate propagates through the CNOT gate, invoking the fourth identity above. Now, the circuit consists of a CNOT gate and two adjacent NOT gates. These adjacent X gates are "cancelled," leading to the final circuit, a single CNOT gate.




In [ ]:
#Build Circuit with 2 Qubits and 3 Gates (1 CNOT and 2 NOT)
circ = QuantumCircuit(2)
circ.x(1)
circ.cx(0, 1)
circ.x(1)
print("Before Optimization:")
print(circ)

#Append "not_propagation" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["not_propagation"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

### Cancel Single Qubit Gates

In addition to *not propgation*, **cancel single qubit gates** is another *propagate-cancel* optimization in VOQC. This also uses cancellation and commutation rules. The cancellation rules are based upon the fact that X, H, and Rz gates are self-cancelling. The commutation rules can be seen in *Figure 6* of Hietala et al. [2019]. However, a key difference between this cancellation and *not_propagation* is that the gates revert back to their original positons if they cannot commute or cancel. 

Here is an example of the self-cancelling nature of an X gate. Multiplying these matrices results in the identity matrix.

<p><center>$\begin{bmatrix}0 & 1\\1 & 0\end{bmatrix}$
$\begin{bmatrix}0 & 1\\1 & 0\end{bmatrix}$
= $\begin{bmatrix}1 & 0\\0 & 1\end{bmatrix}$</center></p>

#### Example 3

Similarly to **not_propagation**, we will append the **VOQC(["cancel_single_qubit_gates"])** to the pass manager to call the "cancel_single_qubit_gates" function in OCaml. 
    
We define a single qubit circuit **circ** with three gates and take advantage of the fact that X gates are self-cancelling. Therefore, we expect that we will end up with a single gate, a z-axis rotation of $\frac {\pi}{2}$. This rotation is a special case rotation, simplifying to an S gate. 




In [ ]:
#Build Circuit with 1 Qubit and 3 Gates (Rz and 2 Hadamard)
circ = QuantumCircuit(1)
circ.rz(pi/2, 0)
circ.h(0)
circ.h(0)
print("Before Optimization:")
print(circ)

#Append "cancel_single_qubit_gates" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["cancel_single_qubit_gates"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

### Cancel Two Qubit Gates

This optimization is almost identical to the *cancel_single_qubit_gates* optimization in the previous section, except for the fact that it applies to two-qubit gates (CX), rather than the X, H, and Rz gates. 

<p><center>$\begin{bmatrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 0 & 1\\0 & 0 & 1 & 0\end{bmatrix}$
$\begin{bmatrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 0 & 1\\0 & 0 & 1 & 0\end{bmatrix}$
= $\begin{bmatrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{bmatrix}$</center></p>

#### Example 4

In our new circuit, we will have an H gate act on qubit 0 and two CNOT gates with control qubit 0 and target qubit 1. Just like earlier, we will add **VOQC(["cancel_two_qubit_gates"])** to the pass manager and run the circuit. 
    
The two CX gates cancel and the final circuit contains just the original hadamard gate.



In [ ]:
#Build Circuit with 2 Qubits and 3 Gates (2 CNOT and 1 Hadamard)
circ = QuantumCircuit(2)
circ.cx(0, 1)
circ.cx(0, 1)
circ.h(0)
print("Before Optimization:")
print(circ)

#Append "cancel_two_qubit_gates" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["cancel_two_qubit_gates"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

### Rotation Merging

In addition to the three optimizations discussed earlier, VOQC also implements two circuit replacement optimizations, where the one series of gates is replaced by an equivalent series. One of these unitary optimizations is **rotation merging**. Unlike the gate cancellations, rotation merging can combine two non-adjacent Rz gates, specifically those in {Rz, CX} subcircuits. Two non-adjacent Rz gates, **Rz(k)** and **Rz(k')**, can be merged into a single Rz gate, **Rz(k+k')**, if they are in the same logical state.

#### Example 5

After building a circuit with 2 adjacent Rz gates, we will append **VOQC(["merge_rotations"])** to **pm**. Once we run this circuit, we notice that the two Rz gates merge into a single S gate because of the special case rotation. 
    

In [ ]:
#Build Circuit with 1 Qubit and 2 Gates (Two Rz)
circ = QuantumCircuit(1)
circ.rz(pi/4, 0)
circ.rz(pi/4, 0)
print("Before Optimization:")
print(circ)

#Append "merge_rotations" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["merge_rotations"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)

### Hadamard Reduction

The final unitary optimization in VOQC is **hadamard reduction**. Hadamard reduction uses the identities in <a href="https://arxiv.org/abs/1912.02250">Figure 7</a> to reduce the number of H gates in a circuit. Although this in its own may not lead to overall gate reduction, the reduction of H gates is crucial towards maximizing the effectiveness of rotation merging as it can maximize the size of the {Rz, CNOT} subcircuits.

#### Example 6

The circuit used to exemplify this transformation is a three-qubit circuit with two hadamard gates and an Rz gate with a $\frac {\pi} {2}$ rotation. To utilize the hadamard reduction optimization, append **hadamard_reduction** to the list in VOQC's argument, **VOQC(["hadamard_reduction"])**. 

After the optimization, the resulting circuit has a two Rz gates with a $\frac {3\pi} {2}$ rotation, the inverse of $\frac {\pi} {2}$, and a hadamard gate.




In [ ]:
#Build Circuit with 1 Qubit and 3 Gates (H and Rz)
circ = QuantumCircuit(1)
circ.h(0)
circ.rz(pi/2, 0)
circ.h(0)

print("Before Optimization:")
print(circ)

#Append "hadamard_reduction" optimization to the Pass Manager
pm = PassManager()
pm.append(VOQC(["hadamard_reduction"]))
new_circ = pm.run(circ)

#Print Optimized Circuit
print("After Optimization:")
print(new_circ)